In [ ]:
import pyodbc
import json
import time
import re
import logging
from uuid import uuid4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openlocationcode import openlocationcode as olc

# Cấu hình
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('scraper.log', encoding='utf-8'), 
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [ ]:
# Chuỗi kết nối SQL Server
connection_string = (
    "DRIVER={SQL Server};"
    "SERVER=LAPTOP-IP1RMDOK\\DATASERVERNGHIA;"
    "DATABASE=restaurants_final;"
    "UID=sa;"
    "PWD=123456;"
    "Trusted_Connection=no;"
)

In [ ]:
# Hàm thiết lập cơ sở dữ liệu
def setup_database():
    """Thiết lập kết nối cơ sở dữ liệu và tạo bảng với kích thước cột phù hợp cho URL.
    
    Returns:
        tuple: (conn, cursor) - Kết nối và con trỏ cơ sở dữ liệu.
    
    Raises:
        pyodbc.Error: Nếu kết nối hoặc tạo bảng thất bại.
    """
    try:
        logger.info("Đang thiết lập kết nối cơ sở dữ liệu...")
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        
        # Tạo bảng restaurants
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'restaurants')
        CREATE TABLE restaurants (
            restaurant_id INT IDENTITY(1,1) PRIMARY KEY,
            restaurant_name NVARCHAR(max),
            url NVARCHAR(max),
            restaurant_type NVARCHAR(255),
            rating_average FLOAT,
            num_of_reviews INT,
            price_level NVARCHAR(50),
            address NVARCHAR(max),
            latitude FLOAT,
            longitude FLOAT,
            phone NVARCHAR(20),
            created_at DATETIME DEFAULT GETDATE()
        )
        ''')

        # Tạo bảng reviews
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'reviews')
        CREATE TABLE reviews (
            review_id INT IDENTITY(1,1) PRIMARY KEY,
            reviewer_name NVARCHAR(500),  
            reviewer_info NVARCHAR(500),  
            rating FLOAT,
            review_time NVARCHAR(100),
            review_text FLOAT,
            service_rating FLOAT,
            food_rating FLOAT,
            atmosphere_rating FLOAT,
            service_type NVARCHAR(100),  
            meal_type NVARCHAR(100), 
            language NVARCHAR(100),
            created_at DATETIME DEFAULT GETDATE(),
            restaurant_id INT,
            CONSTRAINT FK_restaurant_review FOREIGN KEY (restaurant_id) REFERENCES restaurants (restaurant_id)
        )
        ''')

        cursor.execute("SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'restaurants'")
        if cursor.fetchone()[0] == 0:
            raise Exception("Bảng restaurants không được tạo.")
        
        cursor.execute("SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'reviews'")
        if cursor.fetchone()[0] == 0:
            raise Exception("Bảng reviews không được tạo.")
        
        conn.commit()
        logger.info("Thiết lập cơ sở dữ liệu thành công.")
        return conn, cursor
    except Exception as e:
        logger.error(f"Lỗi thiết lập cơ sở dữ liệu: {e}")
        raise

In [ ]:
# Hàm hỗ trợ click
def safe_click(driver, element, wait_time=2, retries=3):
    """Click an phần tử một cách an toàn bằng JavaScript với cơ chế thử lại.
    
    Args:
        driver: WebDriver instance.
        element: WebElement để click.
        wait_time: Thời gian chờ sau khi click (giây).
        retries: Số lần thử lại nếu click thất bại.
    
    Returns:
        bool: True nếu click thành công, False nếu thất bại.
    """
    for attempt in range(retries):
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable(element))
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            driver.execute_script("arguments[0].click();", element)
            time.sleep(wait_time)
            logger.info("Click thành công.")
            return True
        except (ElementClickInterceptedException, TimeoutException) as e:
            logger.warning(f"Click thất bại (lần {attempt + 1}/{retries}): {e}")
            time.sleep(1)
    logger.error(f"Click thất bại sau {retries} lần thử.")
    return False

In [ ]:
# Hàm cuộn và click nút "Xem thêm"
def scroll_and_click_more(driver, scrollable_div, max_scrolls=20):
    """Cuộn và mở rộng nút 'Xem thêm' trong đánh giá.
    
    Args:
        driver: WebDriver instance.
        scrollable_div: Phần tử WebElement chứa danh sách đánh giá.
        max_scrolls: Số lần cuộn tối đa.
    """
    last_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
    scroll_count = 0
    reviews_loaded = 0
    
    while scroll_count < max_scrolls:
        current_reviews = len(driver.find_elements(By.CSS_SELECTOR, "div.jftiEf.fontBodyMedium"))
        logger.info(f"Đã tải {current_reviews} đánh giá.")

        more_buttons = driver.find_elements(By.CSS_SELECTOR, "button.w8nwRe.kyuRq")
        for btn in more_buttons:
            if safe_click(driver, btn, wait_time=0.5):
                time.sleep(0.5)
        
        # Cuộn xuống
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)
        time.sleep(1.5)
        
        # Kiểm tra dữ liệu mới
        new_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
        new_reviews = len(driver.find_elements(By.CSS_SELECTOR, "div.jftiEf.fontBodyMedium"))
        
        if new_height == last_height and new_reviews == reviews_loaded:
            scroll_count += 1
            if scroll_count >= 3:
                logger.info("Không còn đánh giá mới, dừng cuộn.")
                break
        else:
            scroll_count = 0
            logger.info(f"Tải thêm dữ liệu mới. Chiều cao mới: {new_height}, Đánh giá mới: {new_reviews}")
        
        last_height = new_height
        reviews_loaded = new_reviews

In [ ]:
import pandas as pd
# Cào DL lần 2 - import file hiện tại để loại bỏ nhà hàng đã cào
def get_excluded_urls(csv_file='restaurants_new.csv'):
    try:
        df = pd.read_csv(csv_file)
        excluded_urls = df['url'].tolist()
        return excluded_urls
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
        return []

# Gán danh sách excluded_urls
excluded_urls = get_excluded_urls()

In [ ]:
# Hàm cấu hình trình duyệt
def setup_driver():
    """Thiết lập và cấu hình trình duyệt Chrome.
    
    Returns:
        WebDriver: Trình duyệt Chrome đã được cấu hình.
    
    Raises:
        Exception: Nếu thiết lập trình duyệt thất bại.
    """
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.implicitly_wait(10)
        
        # Giả lập vị trí ở Đà Nẵng
        driver.execute_cdp_cmd("Emulation.setGeolocationOverride", {
            "latitude": 16.0544,
            "longitude": 108.2022,
            "accuracy": 100
        })
        
        logger.info("Thiết lập trình duyệt thành công.")
        return driver
    except Exception as e:
        logger.error(f"Lỗi khi thiết lập trình duyệt: {e}")
        raise

In [ ]:
# Hàm nhập dữ liệu chính
def import_scraped_data(driver, conn, cursor, batch_size=10):
    """Nhập dữ liệu từ Google Maps vào cơ sở dữ liệu.
    
    Args:
        driver: WebDriver instance.
        conn: Kết nối cơ sở dữ liệu.
        cursor: Con trỏ cơ sở dữ liệu.
        batch_size: Số lượng nhà hàng xử lý trước khi lưu trữ tạm thời.
    """
    try:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='feed']")))
        divs = element.find_elements(By.XPATH, "./div[position() > 2 and not(@class='TFQHme')]")
        links = [a_tags[0].get_attribute("href") for div in divs if (a_tags := div.find_elements(By.CSS_SELECTOR, "a"))]
        logger.info(f"Tìm thấy {len(links)} nhà hàng để xử lý.")
        
        failed_urls = []
        for i, url in enumerate(links):
            if url in excluded_urls:
                logger.info(f"Nhà hàng {i+1}/{len(links)} ({url}) nằm trong danh sách loại trừ, bỏ qua.")
                continue
            try:
                if url and len(url) > 1999:
                    url = url[:1999]
                
                cursor.execute("SELECT COUNT(*) FROM restaurants WHERE url = ?", (url,))
                if cursor.fetchone()[0] > 0:
                    logger.info(f"Nhà hàng {i+1}/{len(links)} ({url}) đã tồn tại, bỏ qua.")
                    continue
                
                logger.info(f"Đang xử lý nhà hàng {i+1}/{len(links)}: {url}")
                driver.get(url)
                
                # Trích xuất dữ liệu nhà hàng
                # Lấy tên nhà hàng
                restaurant_name = None
                try:
                    restaurant_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[1]/h1'))).text
                except:
                    logger.warning("Không tìm thấy tên nhà hàng.")
                
                # Lấy mức giá
                price = None
                try:
                    price = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/span/span/span/span[2]/span/span').text
                except:
                    logger.warning("Không tìm thấy mức giá.")
                
                # Lấy đánh giá
                rating_average = None
                try:
                    rating_average = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]'))).text
                except:
                    logger.warning("Không tìm thấy đánh giá.")
                
                # Lấy số lượng đánh giá    
                num_of_reviews = None
                try:
                    reviews_elem = driver.find_element(By.CSS_SELECTOR, "div.F7nice span[aria-label*='reviews']")
                    reviews_text = reviews_elem.get_attribute("aria-label").split()[0]
                    reviews_text = reviews_text.replace(",", "")
                    num_of_reviews = int(reviews_text)
                except:
                    logger.warning("Không tìm thấy số lượng đánh giá.")
                
                # Lấy số điện thoại
                phone = None
                try:
                    phone_elem = driver.find_element(By.CSS_SELECTOR, 'button[data-item-id^="phone:tel"]')
                    phone = phone_elem.get_attribute('data-item-id').replace("phone:tel:", "")
                except:
                    logger.warning("Không tìm thấy số điện thoại.")
                
                # Lấy loại nhà hàng
                restaurant_type = None
                try:
                    type_elems = driver.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[2]/span[1]/span/button')
                    restaurant_type = ', '.join([elem.text for elem in type_elems])
                except:
                    logger.warning("Không tìm thấy loại nhà hàng.")
                
                address = None
                try:
                    address_xpaths = [
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[3]/button/div/div[2]/div[1]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[11]/div[3]/button/div/div[2]/div[1]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[13]/div[3]/button/div/div[2]/div[1]'
                    ]
                    for xpath in address_xpaths:
                        try:
                            address_element = driver.find_element(By.XPATH, xpath)
                            address = address_element.text.strip()
                            if address:
                                break
                        except NoSuchElementException:
                            continue
                except:
                    logger.warning("Không tìm thấy địa chỉ.")

                # Lấy Plus Code
                plus_code = None
                latitude = None
                longitude = None
                try:
                    plus_code_elem = driver.find_element(By.XPATH, "//button[contains(@aria-label, 'Plus code') and @class='CsEnBe']")
                    aria_label = plus_code_elem.get_attribute("aria-label")

                    if aria_label:
                        plus_code = aria_label.replace("Plus code: ", "").strip()
                        code = plus_code.split()[0]

                        if olc.isValid(code):
                            try:
                                reference_latitude = 16.067
                                reference_longitude = 108.220
                                full_code = olc.recoverNearest(code, reference_latitude, reference_longitude)
                                decoded = olc.decode(full_code)
                                latitude = decoded.latitudeCenter
                                longitude = decoded.longitudeCenter
                                print(f"Đã chuyển đổi Plus Code {code}: Latitude = {latitude}, Longitude = {longitude}")
                            except ValueError as e:
                                print(f"Lỗi khi giải mã Plus Code {code}: {e}")
                        else:
                            print(f"Plus Code không hợp lệ: {code} (từ chuỗi: {plus_code})")
                    else:
                        print("Không tìm thấy Plus Code.")

                except NoSuchElementException:
                    print("Không tìm thấy phần tử Plus Code.")
                except Exception as e:
                    print(f"Lỗi khi lấy Plus Code: {e}")

                # Chuẩn hóa dữ liệu trước khi chèn
                rating_average = rating_average if rating_average is not None else 0.0
                phone = phone if phone and len(phone) <= 20 else None
                restaurant_name = restaurant_name if restaurant_name and len(restaurant_name) <= 500 else restaurant_name[:500] if restaurant_name else None
                restaurant_type = restaurant_type if restaurant_type and len(restaurant_type) <= 500 else restaurant_type[:500] if restaurant_type else None
                address = address if address and len(address) <= 1000 else address[:1000] if address else None
                price = price if price and len(price) <= 50 else price[:50] if price else None

                # Chèn dữ liệu nhà hàng
                try:
                    cursor.execute('''
                        INSERT INTO restaurants (url, restaurant_name, restaurant_type, rating_average, num_of_reviews, phone, price_level, address, latitude, longitude)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (url, restaurant_name, restaurant_type, rating_average, num_of_reviews, phone, price, address, latitude, longitude,))
                    try:
                        conn.commit()
                    except Exception as e:
                        logger.error(f"Lỗi khi commit dữ liệu nhà hàng: {e}")
                        conn.rollback()
                except Exception as e:
                    logger.error(f"Lỗi khi chèn dữ liệu nhà hàng: {e}")
                    conn.rollback()
                
                cursor.execute("SELECT IDENT_CURRENT('restaurants')")
                restaurant_id = int(cursor.fetchone()[0])
                logger.info(f"Đã chèn nhà hàng với ID: {restaurant_id}")
                
                # Lấy đánh giá
                try:
                    reviews_tab = driver.find_elements(By.XPATH, "//div[text()='Reviews']")
                    if not reviews_tab:
                        logger.warning("Không tìm thấy tab đánh giá.")
                        continue
                    
                    safe_click(driver, reviews_tab[0])
                    scrollable_divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf")))
                    if scrollable_divs:
                        scroll_and_click_more(driver, scrollable_divs[0])
                    
                    review_containers = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf.fontBodyMedium")
                    logger.info(f"Tìm thấy {len(review_containers)} đánh giá.")
                    
                    for container in review_containers:
                        try:
                            reviewer_name = container.find_element(By.CSS_SELECTOR, "div.d4r55").text or None
                            reviewer_info = container.find_element(By.CSS_SELECTOR, "div.RfnDt").text or None
                            
                            rating_value = None
                            try:
                                rating_elem = container.find_element(By.CSS_SELECTOR, "span[aria-label]")
                                rating_text = rating_elem.get_attribute('aria-label').split()[0]
                                rating_value = float(rating_text.replace(',', '.'))
                            except:
                                pass
                            
                            review_time = container.find_element(By.CSS_SELECTOR, "span.rsqaWe").text or None
                            review_text = container.find_element(By.CSS_SELECTOR, "span.wiI7pd").text or None
                            
                            language = None
                            try:
                                language_elem = container.find_element(By.CSS_SELECTOR, "div.oqftme")
                                text = language_elem.text
                                if "(" in text and ")" in text:
                                    language = text.split("(")[-1].replace(")", "").strip()
                            except:
                                pass
                            
                            rating_elements = container.find_elements(By.CSS_SELECTOR, "span.RfDO5c")
                            service_rating = food_rating = atmosphere_rating = service_type = meal_type = None
                            
                            for elem in rating_elements:
                                text = elem.text.strip()
                                if "Service:" in text:
                                    service_rating = text.split(":")[-1].strip()
                                elif "Food:" in text:
                                    food_rating = text.split(":")[-1].strip()
                                elif "Atmosphere:" in text:
                                    atmosphere_rating = text.split(":")[-1].strip()
                                elif text in ["Dine in", "Takeout"]:
                                    service_type = text
                                elif text in ["Breakfast", "Lunch", "Dessert", "Brunch", "Dinner", "Seating"]:
                                    meal_type = text
                            
                            # Xử lý trường hợp review_text không có nội dung
                            if not review_text or review_text.strip() == "":
                                if rating_value is not None:
                                    review_text = f"Rated {rating_value}"
                                    logger.info(f"Đã gán review_text thành 'Rated {rating_value}' vì không có nội dung đánh giá.")
                                else:
                                    logger.info("Không có nội dung đánh giá và rating_value, giữ review_text là None.")
                            
                            reviewer_name = reviewer_name if reviewer_name and len(reviewer_name) <= 500 else reviewer_name[:500] if reviewer_name else None
                            reviewer_info = reviewer_info if reviewer_info and len(reviewer_info) <= 500 else reviewer_info[:500] if reviewer_info else None
                            review_time = review_time if review_time and len(review_time) <= 100 else review_time[:100] if review_time else None
                            language = language if language and len(language) <= 100 else language[:100] if language else None
                            
                            cursor.execute('''
                                INSERT INTO reviews (restaurant_id, reviewer_name, reviewer_info, rating, review_time, 
                                            review_text, service_rating, food_rating, atmosphere_rating, 
                                            service_type, meal_type, language)
                                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                            ''', (restaurant_id, reviewer_name, reviewer_info, rating_value, review_time, 
                                  review_text, service_rating, food_rating, atmosphere_rating, 
                                  service_type, meal_type, language))
                        except Exception as e:
                            logger.warning(f"Lỗi khi xử lý đánh giá: {e}")
                            continue
                    
                    try:
                        conn.commit()
                    except Exception as e:
                        logger.error(f"Lỗi khi commit dữ liệu đánh giá: {e}")
                        conn.rollback()
                    logger.info("Đã nhập đánh giá thành công.")
                except Exception as e:
                    logger.error(f"Lỗi khi xử lý đánh giá cho nhà hàng {restaurant_name}: {e}")
                    conn.rollback()
                
                if (i + 1) % batch_size == 0:
                    logger.info(f"Đã xử lý {i + 1} nhà hàng, lưu trữ tạm thời.")
                    conn.commit()
                    driver.quit()
                    driver = setup_driver()
            except Exception as e:
                logger.error(f"Lỗi khi xử lý nhà hàng {i+1}/{len(links)} ({url}): {e}")
                failed_urls.append(url)
                conn.rollback()
                continue
        
        if failed_urls:
            logger.warning(f"Các URL thất bại: {failed_urls}")
    except Exception as e:
        logger.error(f"Lỗi khi tìm liên kết nhà hàng: {e}")

In [ ]:
# Hàm cuộn để tải tất cả nhà hàng
def scroll_until_end(driver, scrollable_div, max_attempts=8):
    """Cuộn xuống cho đến khi tải hết tất cả nhà hàng.
    
    Args:
        driver: WebDriver instance.
        scrollable_div: Phần tử WebElement chứa danh sách nhà hàng.
        max_attempts: Số lần thử tối đa khi không có dữ liệu mới.
    """
    last_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
    no_change_count = 0
    total_restaurants = 0
    last_restaurant_count = 0

    while no_change_count < max_attempts:
        restaurants = driver.find_elements(By.CSS_SELECTOR, ".Nv2PK.THOPZb.CpccDe")
        total_restaurants = len(restaurants)
        logger.info(f"Tìm thấy {total_restaurants} nhà hàng (mới: {total_restaurants - last_restaurant_count})")
        
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)
        time.sleep(2)  
        
        new_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
        new_restaurant_count = len(driver.find_elements(By.CSS_SELECTOR, ".Nv2PK.THOPZb.CpccDe"))

        if new_height == last_height and new_restaurant_count == last_restaurant_count:
            no_change_count += 1
            logger.info(f"Không tìm thấy dữ liệu mới: {no_change_count}/{max_attempts}")
        else:
            no_change_count = 0
            logger.info(f"Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: {new_height}")
        
        last_height = new_height
        last_restaurant_count = new_restaurant_count
    
    logger.info(f"Hoàn thành cuộn! Tìm thấy tổng cộng {total_restaurants} nhà hàng.")

In [ ]:
# %%
# Thực thi chính
if __name__ == "__main__":
    driver = None
    conn = None
    
    try:
        logger.info("Thiết lập kết nối cơ sở dữ liệu...")
        conn, cursor = setup_database()
        
        logger.info("Khởi tạo trình duyệt...")
        driver = setup_driver()
        
        url = "https://www.google.com/maps/search/Restaurants+in+Da+Nang"
        logger.info(f"Mở URL: {url}")
        driver.get(url)
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='main']")))
        
        logger.info("Tìm danh sách nhà hàng...")
        try:
            wait = WebDriverWait(driver, 20)
            scrollable_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='feed']")))
            
            logger.info("Bắt đầu cuộn để tải thêm nhà hàng...")
            scroll_until_end(driver, scrollable_div)
            
            logger.info("Bắt đầu nhập dữ liệu nhà hàng...")
            import_scraped_data(driver, conn, cursor)
            logger.info("Nhập dữ liệu thành công!")
        except TimeoutException as e:
            logger.error(f"Không thể tìm thấy danh sách nhà hàng: {e}")
            raise
        
    except Exception as e:
        logger.error(f"Lỗi trong quá trình thực thi: {e}")
    finally:
        if conn:
            try:
                conn.close()
                logger.info("Đã đóng kết nối cơ sở dữ liệu.")
            except:
                logger.warning("Lỗi khi đóng kết nối cơ sở dữ liệu.")
        
        if driver:
            driver.quit()
            logger.info("Đã đóng trình duyệt.")

2025-05-01 22:26:42,008 - INFO - Thiết lập kết nối cơ sở dữ liệu...
2025-05-01 22:26:42,008 - INFO - Đang thiết lập kết nối cơ sở dữ liệu...
2025-05-01 22:26:42,906 - INFO - Thiết lập cơ sở dữ liệu thành công.
2025-05-01 22:26:42,906 - INFO - Khởi tạo trình duyệt...
2025-05-01 22:26:42,906 - INFO - ====== WebDriver manager ======
2025-05-01 22:26:43,909 - INFO - Get LATEST chromedriver version for google-chrome
2025-05-01 22:26:44,033 - INFO - Get LATEST chromedriver version for google-chrome
2025-05-01 22:26:44,158 - INFO - Driver [C:\Users\acer\.wdm\drivers\chromedriver\win64\135.0.7049.114\chromedriver-win32/chromedriver.exe] found in cache
2025-05-01 22:26:45,516 - INFO - Thiết lập trình duyệt thành công.
2025-05-01 22:26:45,518 - INFO - Mở URL: https://www.google.com/maps/search/Restaurants+in+Da+Nang
2025-05-01 22:26:47,816 - INFO - Tìm danh sách nhà hàng...
2025-05-01 22:26:47,832 - INFO - Bắt đầu cuộn để tải thêm nhà hàng...
2025-05-01 22:26:47,856 - INFO - Tìm thấy 8 nhà hàng 

Đã chuyển đổi Plus Code 367H+MV: Latitude = 16.064187500000003, Longitude = 108.2296875


2025-05-01 22:48:29,822 - INFO - Click thành công.
2025-05-01 22:48:37,270 - INFO - Đã tải 10 đánh giá.
2025-05-01 22:48:37,833 - INFO - Click thành công.
2025-05-01 22:48:38,866 - INFO - Click thành công.
2025-05-01 22:48:39,896 - INFO - Click thành công.
2025-05-01 22:48:40,934 - INFO - Click thành công.
2025-05-01 22:48:41,966 - INFO - Click thành công.
2025-05-01 22:48:43,007 - INFO - Click thành công.
2025-05-01 22:48:44,041 - INFO - Click thành công.
2025-05-01 22:48:45,072 - INFO - Click thành công.
2025-05-01 22:48:46,104 - INFO - Click thành công.
2025-05-01 22:48:47,141 - INFO - Click thành công.
2025-05-01 22:48:49,161 - INFO - Tải thêm dữ liệu mới. Chiều cao mới: 19454, Đánh giá mới: 30
2025-05-01 22:48:49,169 - INFO - Đã tải 30 đánh giá.
2025-05-01 22:48:49,713 - INFO - Click thành công.
2025-05-01 22:48:50,778 - INFO - Click thành công.
2025-05-01 22:48:51,842 - INFO - Click thành công.
2025-05-01 22:48:52,890 - INFO - Click thành công.
2025-05-01 22:48:53,928 - INFO - Cl

Đã chuyển đổi Plus Code 368F+9G: Latitude = 16.065937499999997, Longitude = 108.2238125


2025-05-01 23:14:26,865 - INFO - Click thành công.
2025-05-01 23:14:36,943 - INFO - Đã tải 0 đánh giá.
2025-05-01 23:14:44,897 - INFO - Click thành công.
2025-05-01 23:14:45,950 - INFO - Click thành công.
2025-05-01 23:14:46,986 - INFO - Click thành công.
2025-05-01 23:14:48,020 - INFO - Click thành công.
2025-05-01 23:14:49,060 - INFO - Click thành công.
2025-05-01 23:14:50,094 - INFO - Click thành công.
2025-05-01 23:14:51,128 - INFO - Click thành công.
2025-05-01 23:14:52,164 - INFO - Click thành công.
2025-05-01 23:14:53,203 - INFO - Click thành công.
2025-05-01 23:14:54,244 - INFO - Click thành công.
2025-05-01 23:14:55,278 - INFO - Click thành công.
2025-05-01 23:14:56,315 - INFO - Click thành công.
2025-05-01 23:14:57,349 - INFO - Click thành công.
2025-05-01 23:14:58,385 - INFO - Click thành công.
2025-05-01 23:14:59,417 - INFO - Click thành công.
2025-05-01 23:15:00,453 - INFO - Click thành công.
2025-05-01 23:15:01,496 - INFO - Click thành công.
2025-05-01 23:15:03,518 - INFO

...

2025-05-02 00:59:25,922 - INFO - Click thành công.
2025-05-02 00:59:35,938 - INFO - Đã tải 0 đánh giá.
2025-05-02 00:59:43,361 - INFO - Click thành công.
2025-05-02 00:59:44,398 - INFO - Click thành công.
2025-05-02 00:59:45,428 - INFO - Click thành công.
2025-05-02 00:59:46,462 - INFO - Click thành công.
2025-05-02 00:59:47,504 - INFO - Click thành công.
2025-05-02 00:59:48,537 - INFO - Click thành công.
2025-05-02 00:59:49,584 - INFO - Click thành công.
2025-05-02 00:59:50,647 - INFO - Click thành công.
2025-05-02 00:59:51,699 - INFO - Click thành công.
2025-05-02 00:59:52,732 - INFO - Click thành công.
2025-05-02 00:59:54,756 - INFO - Tải thêm dữ liệu mới. Chiều cao mới: 23627, Đánh giá mới: 30
2025-05-02 00:59:54,771 - INFO - Đã tải 30 đánh giá.
2025-05-02 00:59:55,377 - INFO - Click thành công.
2025-05-02 00:59:56,447 - INFO - Click thành công.
2025-05-02 00:59:57,485 - INFO - Click thành công.
2025-05-02 00:59:58,522 - INFO - Click thành công.
2025-05-02 00:59:59,562 - INFO - Cli

Đã chuyển đổi Plus Code 26XX+G2: Latitude = 16.048812499999997, Longitude = 108.2475625


2025-05-02 01:42:38,365 - INFO - Click thành công.
2025-05-02 01:42:48,384 - INFO - Đã tải 0 đánh giá.
2025-05-02 01:42:56,296 - INFO - Click thành công.
2025-05-02 01:42:57,329 - INFO - Click thành công.
2025-05-02 01:42:58,367 - INFO - Click thành công.
2025-05-02 01:42:59,399 - INFO - Click thành công.
2025-05-02 01:43:00,465 - INFO - Click thành công.
2025-05-02 01:43:01,520 - INFO - Click thành công.
2025-05-02 01:43:02,573 - INFO - Click thành công.
2025-05-02 01:43:03,613 - INFO - Click thành công.
2025-05-02 01:43:04,652 - INFO - Click thành công.
2025-05-02 01:43:05,686 - INFO - Click thành công.
2025-05-02 01:43:06,715 - INFO - Click thành công.
2025-05-02 01:43:07,752 - INFO - Click thành công.
2025-05-02 01:43:08,784 - INFO - Click thành công.
2025-05-02 01:43:09,834 - INFO - Click thành công.
2025-05-02 01:43:11,847 - INFO - Tải thêm dữ liệu mới. Chiều cao mới: 24096, Đánh giá mới: 30
2025-05-02 01:43:11,862 - INFO - Đã tải 30 đánh giá.
2025-05-02 01:43:12,404 - INFO - Cli

Đã chuyển đổi Plus Code 369V+H2: Latitude = 16.068937499999997, Longitude = 108.2425625


2025-05-02 02:25:58,079 - INFO - Click thành công.
2025-05-02 02:26:08,136 - INFO - Đã tải 0 đánh giá.
2025-05-02 02:26:16,247 - INFO - Click thành công.
2025-05-02 02:26:17,288 - INFO - Click thành công.
2025-05-02 02:26:18,321 - INFO - Click thành công.
2025-05-02 02:26:19,379 - INFO - Click thành công.
2025-05-02 02:26:20,411 - INFO - Click thành công.
2025-05-02 02:26:21,446 - INFO - Click thành công.
2025-05-02 02:26:22,479 - INFO - Click thành công.
2025-05-02 02:26:23,518 - INFO - Click thành công.
2025-05-02 02:26:24,549 - INFO - Click thành công.
2025-05-02 02:26:25,593 - INFO - Click thành công.
2025-05-02 02:26:27,605 - INFO - Tải thêm dữ liệu mới. Chiều cao mới: 19552, Đánh giá mới: 30
2025-05-02 02:26:27,618 - INFO - Đã tải 30 đánh giá.
2025-05-02 02:26:28,164 - INFO - Click thành công.
2025-05-02 02:26:29,231 - INFO - Click thành công.
2025-05-02 02:26:30,288 - INFO - Click thành công.
2025-05-02 02:26:31,324 - INFO - Click thành công.
2025-05-02 02:26:32,364 - INFO - Cli